In [ ]:
"""
Created on Fri Jan 27 15:18 2023

Look at the results of the test of the NN on Smith data

Author: @claraburgard

"""

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
%matplotlib qt5

DEFINE OPTIONS

In [ ]:
mod_size =  'large' #'mini', 'small', 'medium', 'large', 'extra_large'
TS_opt = 'extrap' # extrap, whole, thermocline
norm_method =  'std' # std, interquart, minmax
exp_name = 'newbasic2' #'allbutconstants' #'onlyTSdraftandslope' #'TSdraftbotandiceddandwcd' #'onlyTSisfdraft' #'TSdraftbotandiceddandwcdreldGL' #TSdraftslopereldGL

In [ ]:
#isf_dim = [10,11,12,13,18,22,23,25,30,31,33,39,40,42,43,44,45,47,51,55,58,61,65,66,69,70,71,73,75]

In [ ]:
inputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/'
outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/'

NN results

In [ ]:
res_1D_mods_list = []
for mod_size in ['xsmall96', 'small', 'large']:
    res_1D_run_list = []
    for nrun in ['bf663','bi646']:
        outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/SMITH_'+nrun+'/'
        res_1D_all_list = []
        for yy in range(1980, 1980 + 60):
            res_1D_yy = xr.open_dataset(outputpath_melt_nn + 'evalmetrics_1D_'+mod_size+'_'+exp_name+'_ensmean_'+TS_opt+'_norm'+norm_method+'_'+str(yy)+'_'+nrun+'.nc')
            res_1D_all_list.append(res_1D_yy.assign_coords({'time': yy}))
        res_1D_all_xr = xr.concat(res_1D_all_list, dim='time')
        res_1D_run_list.append(res_1D_all_xr.assign_coords({'nemo_run': nrun}))
    res_1D_run_all = xr.concat(res_1D_run_list, dim='nemo_run')
    res_1D_mods_list.append(res_1D_run_all.assign_coords({'mod_size': mod_size}))
res_1D_mods_all = xr.concat(res_1D_mods_list, dim='mod_size')

In [ ]:
diff_Gt = (res_1D_mods_all['predicted_melt'] - res_1D_mods_all['reference_melt']).sel(metrics='Gt').where(res_1D_run_all.Nisf > 9, drop=True)

In [ ]:
diff_Gt

In [ ]:
np.sqrt(((diff_Gt)**2).mean(['time','Nisf']))

In [ ]:
np.sqrt((diff_Gt.sel(time=range(1980+30,1980+60))**2).mean(['time','Nisf']))

In [ ]:
np.sqrt((diff_Gt.sel(time=range(1980,1980+30))**2).mean(['time','Nisf']))

FIRST IMPRESSION - RMSE

In [ ]:
np.sqrt((((melt_1D_bi646['predicted_melt'] - melt_1D_bi646['reference_melt']).sel(metrics='Gt').where(melt_1D_bi646.Nisf > 11, drop=True))**2).mean()).values

In [ ]:
(melt_1D_bi646['predicted_melt'] - melt_1D_bi646['reference_melt']).sel(metrics='Gt').plot() #.where(melt_1D_bi646.Nisf > 10, drop=True)

In [ ]:
np.sqrt((((melt_1D_bi646['predicted_melt'] - melt_1D_bi646['reference_melt']).sel(metrics='box1'))**2).mean()).values

In [ ]:
yend = 35

mean_10_list = []
for tstart in range(0,yend,10):
    mean_10yy = melt_1D_bi646.sel(time=range(tstart,min(tstart+10,yend))).mean('time')
    mean_10_list.append(mean_10yy.assign_coords({'time': tstart}))
mean_10_ds_bi646 = xr.concat(mean_10_list, dim='time')

In [ ]:
np.sqrt((((mean_10_ds_bi646['predicted_melt'] - mean_10_ds_bi646['reference_melt']).sel(metrics='box1'))**2).mean()).values

In [ ]:
np.sqrt((((melt_1D_bf663['predicted_melt'] - melt_1D_bf663['reference_melt']).sel(metrics='Gt'))**2).mean()).values

In [ ]:
(melt_1D_bf663['predicted_melt'] - melt_1D_bf663['reference_melt']).sel(metrics='Gt').plot()

In [ ]:
yend = 40

mean_10_list = []
for tstart in range(0,yend,10):
    mean_10yy = melt_1D_bf663.sel(time=range(tstart,min(tstart+10,yend))).mean('time')
    mean_10_list.append(mean_10yy.assign_coords({'time': tstart}))
mean_10_ds_bf663 = xr.concat(mean_10_list, dim='time')

In [ ]:
np.sqrt((((mean_10_ds_bf663['predicted_melt'] - mean_10_ds_bf663['reference_melt']).sel(metrics='box1'))**2).mean()).values

In [ ]:
np.sqrt((((melt_1D_bf663['predicted_melt'] - melt_1D_bf663['reference_melt']).sel(metrics='box1'))**2).mean()).values

In [ ]:
abs((melt_1D_bi646['predicted_melt'] - melt_1D_bi646['reference_melt']).sel(metrics='Gt')).mean('time').plot()

LOOK INTO MORE DETAILS

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'SMITH_bi646/nemo_5km_isf_masks_and_info_and_distance_oneFRIS_1980.nc').drop('time')
#file_isf = file_isf_orig.sel(Nisf=isf_dim)

In [ ]:
def plot_scatter_all_isf(param_melt_tuned, target_Gt_yr, file_isf):
    
    regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
    colors = ['deepskyblue','brown','red','orange','limegreen','seagreen','magenta']
    symbol = ['o','v','>','p','*','s','<','^','X','d']
    
    f = plt.figure()
    f.set_size_inches(8.25*1.5, 8.25*1.5)

    ax={}

    i = 0
    nn=0
    ii=0
    marker_symbol = np.zeros((len(regions))).astype(int)
    for rr,reg in enumerate(regions):
        k = 0
        subset_isf = file_isf.Nisf.where(file_isf['region']==reg,drop=True)
        for kisf in tqdm(subset_isf.Nisf):
            
            ax[i] = f.add_subplot(6,6,i+1)
            
            for mm,mmod in enumerate(param_melt_tuned.mod_size):
                marker_type = symbol[k]


                #print(i)

                x_axis = param_melt_tuned.sel(Nisf=kisf, mod_size=mmod)
                y_axis = target_Gt_yr.sel(Nisf=kisf, mod_size=mmod)


                ax[i].scatter(x_axis, 
                            y_axis,
                            s=10, c=colors[mm], marker=marker_type, edgecolors='None',
                            rasterized=True)

            
            min_xy = min(x_axis.min(),y_axis.min())
            max_xy = max(x_axis.max(),y_axis.max())
            onetooneline = np.arange(min_xy,max_xy)
            ax[i].set_xlim(min_xy-5,max_xy+5)
            ax[i].set_ylim(min_xy-5,max_xy+5)
            ax[i].plot(onetooneline,onetooneline,'k-')
            #if i == 0 or i==3:
            #    ax[i].set_ylabel('Target melt [Gt/yr]')
            #if i == 3 or i==4:
            #    ax[i].set_xlabel('Predicted melt [Gt/yr]')



            ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))

            i = i+1
            k = k+1


    f.tight_layout()
    sns.despine()

    return f

In [ ]:
plot_scatter_all_isf(melt_1D_bi646['predicted_melt'].sel(metrics='Gt'), melt_1D_bi646['reference_melt'].sel(metrics='Gt'), file_isf)

In [ ]:
Gt_stuff = res_1D_mods_all.sel(metrics='box1',nemo_run='bi646')

In [ ]:
plot_scatter_all_isf(Gt_stuff['predicted_melt'], Gt_stuff['reference_melt'], file_isf_orig.sel(Nisf=res_1D_mods_all.Nisf))
